# Postprocess PIV results (simple calibration)

In [ ]:
import h5py
import numpy as np

In [ ]:
from fluidimage import path_image_samples

In [ ]:
from fluidimage.calibration import Calibration2DSimple

In [ ]:
path_base = path_image_samples / 'Milestone'
path_piv = path_base / 'piv_0000a-b.h5'

## Create the calibration object

In [ ]:
with h5py.File(path_piv) as file:
    shape_im = file['couple/shape_images'].value

point0 = (10, 20)
point1 = (10, 10)

calib = Calibration2DSimple(point0, point1, distance=0.01, aspect_ratio_pixel=1.0,
                            shape_image=shape_im, point_origin=[i//2 for i in shape_im])

In [ ]:
print(calib.yphys1pixel)

## Application to synthetic data

We can try the calibration object on synthetic data:

In [ ]:
ixs = np.linspace(0, 100, 11)
iys = np.linspace(0, 100, 11)
xphys, yphys = calib.pix2phys(ixs, iys)
print(xphys, yphys, sep='\n')

In [ ]:
dxs = np.random.randint(-5, 5, ixs.size)
dys = np.random.randint(-5, 5, ixs.size)
xphys, yphys, dxphys, dyphys = calib.displ2phys(ixs, iys, dxs, dys)
print(xphys, yphys, dxphys, dyphys, sep='\n')

## Application to real data

In [ ]:
with h5py.File(path_piv) as file:
    deltaxs_final = file['piv1/deltaxs_final'][:]
    deltays_final = file['piv1/deltays_final'][:]
    ixvecs_final = file['piv1/ixvecs_final'][:]
    iyvecs_final = file['piv1/iyvecs_final'][:]

In [ ]:
xphys, yphys, dxphys, dyphys = calib.displ2phys(
    ixvecs_final, iyvecs_final, deltaxs_final, deltays_final)

In [ ]:
print(xphys, yphys, sep='\n')

## 2d grid

We need to produce a good grid in the physical space to interpolate the data:

In [ ]:
from fluidimage.postproc.piv import get_grid_pixel_from_piv_file

xs1d, ys1d = get_grid_pixel_from_piv_file(path_piv)

print(xs1d, ys1d, sep='\n')

In [ ]:
print(len(xs1d), len(ys1d))

In [ ]:
xs2d, ys2d = np.meshgrid(xs1d, ys1d)

assert xs2d.shape == ys2d.shape
print(xs2d.shape)

In [ ]:
print(xs2d)

In [ ]:
print(ys2d)

In [ ]:
xs2dphys, ys2dphys = calib.pix2phys(xs2d, ys2d)

In [ ]:
print(xs2dphys)

In [ ]:
print(ys2dphys)

We define a function to interpolate the data on the grid

In [ ]:
from scipy.interpolate import griddata

def gridd(delta):
        delta_grid = griddata((yphys, xphys), delta, (ys2dphys, xs2dphys), method='cubic')
        return delta_grid.astype('float32')

In [ ]:
dxphys_grid = gridd(dxphys)
dyphys_grid = gridd(dyphys)

In [ ]:
print(dxphys_grid)

We can finally produce PIVOnGrid objects...